In [15]:
import dlib
import cv2
import numpy as np
import os
def get_face_rect(image):
    detector = dlib.get_frontal_face_detector()
    faces = detector(image)
    if len(faces) == 0:
        h, w = image.shape[:2]
        return dlib.rectangle(0, 0, w, h)
    return faces[0]

def draw_keypoints(image, landmarks, box=None, color=(0, 255, 0), radius=2, thickness=-1):
    image_out = image.copy()
    for (x, y) in landmarks:
        cv2.circle(image_out, (int(x), int(y)), radius, color, thickness)
    if box is not None:
        cv2.rectangle(image_out, (box.left(), box.top()), (box.right(), box.bottom()), color, 2)
    return image_out

def read_pts(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    landmarks = []

    for line in lines[3:-1]:
        x, y = map(float, line.strip().split())
        landmarks.append([x, y])
    
    return np.array(landmarks)

In [38]:
len(read_pts('data/Menpo/train/aflw__face_39822.pts'))

39

In [39]:
len(read_pts('data/Menpo/train/fddb__image2661_1.pts'))

68

In [47]:
count1, count2 = 0, 0
folder = 'data/Menpo/test'
for pts_path in os.listdir(folder):
    pts = os.path.join(folder, pts_path)
    if not pts.endswith('.pts'):
        continue
    count1 += 1
    a = len(read_pts(pts))
    if (len(read_pts(pts)) == 68):
        count2 += 1
count1, count2

(898, 661)

In [34]:
files = []
count1, count2 = 0, 0
folder = 'data/300W/train'
for pts_path in os.listdir(folder):
    pts = os.path.join(folder, pts_path)
    if not pts.endswith('.pts'):
        continue
    count1 += 1
    if len(read_pts(pts) == 68):
        count2 += 1
count1, count2

(3292, 3292)

In [48]:
len(True)

TypeError: object of type 'bool' has no len()